# Continuous Optimization

In this tutorial we will go into some of the details of optimizing a Euclidean landscape in continuous space.

# Objective

The general objective under this optimization scheme is 